In [2]:
%reload_ext autoreload
%autoreload 2

## Augmentation Operations

Currently there's only 1 (somewhat) stable augmentation in Recon, replacing an Entity label with a random choice from a list of options

In [3]:
from recon.augmentation import ent_label_sub
from recon.dataset import Dataset
from recon.types import Example, Span

ModuleNotFoundError: No module named 'recon'

In [18]:
example = Example(
    text="This is a first sentence with entity. This is an entity in the 2nd sentence.",
    spans=[
        Span(text="entity", start=30, end=36, label="ENTITY"),
        Span(text="entity", start=49, end=55, label="ENTITY"),
    ],
)

ds = Dataset("test_dataset", data=[example])
ds.apply_("recon.v1.augment.ent_label_sub", label="ENTITY", subs=["new entity"], sub_prob=1.0)

assert len(ds) == 2

assert ds.data[0] == example
assert ds.data[1] == Example(
    text="This is a first sentence with new entity. This is an new entity in the 2nd sentence.",
    spans=[
        Span(text="new entity", start=30, end=40, label="ENTITY"),
        Span(text="new entity", start=53, end=63, label="ENTITY"),
    ],
)

100%|██████████| 1/1 [00:00<00:00, 547.20it/s]

=> Applying operation 'recon.v1.augment.ent_label_sub' to dataset 'test_dataset'
OPERATION RESULT RETURNED LIST [Example(text='This is a first sentence with entity. This is an entity in the 2nd sentence.', spans=[Span(text='entity', start=30, end=36, label='ENTITY', token_start=None, token_end=None, kb_id=None), Span(text='entity', start=49, end=55, label='ENTITY', token_start=None, token_end=None, kb_id=None)], tokens=None, meta={}, formatted=True), Example(text='This is a first sentence with new entity. This is an new entity in the 2nd sentence.', spans=[Span(text='new entity', start=30, end=40, label='ENTITY', token_start=None, token_end=None, kb_id=None), Span(text='new entity', start=53, end=63, label='ENTITY', token_start=None, token_end=None, kb_id=None)], tokens=None, meta={}, formatted=True)]
✔ Completed operation 'recon.v1.augment.ent_label_sub'


In [7]:
ent_label_sub(example, "ENTITY", ["new entity"], sub_prob=1.0)

[Example(text='This is a first sentence with entity. This is an entity in the 2nd sentence.', spans=[Span(text='new entity', start=30, end=40, label='ENTITY', token_start=None, token_end=None, kb_id=None), Span(text='new entity', start=53, end=63, label='ENTITY', token_start=None, token_end=None, kb_id=None)], tokens=None, meta={}, formatted=True),
 Example(text='This is a first sentence with new entity. This is an new entity in the 2nd sentence.', spans=[Span(text='new entity', start=30, end=40, label='ENTITY', token_start=None, token_end=None, kb_id=None), Span(text='new entity', start=53, end=63, label='ENTITY', token_start=None, token_end=None, kb_id=None)], tokens=None, meta={}, formatted=True)]

In [10]:
ds = Dataset("test_dataset", data=[example])
ds.apply_("recon.v1.augment.ent_label_sub", label="ENTITY", subs=["new entity"], sub_prob=1.0)

100%|██████████| 1/1 [00:00<00:00, 184.54it/s]

=> Applying operation 'recon.v1.augment.ent_label_sub' to dataset 'test_dataset'
✔ Completed operation 'recon.v1.augment.ent_label_sub'


In [17]:
ds.data

[Example(text='This is a first sentence with entity. This is an entity in the 2nd sentence.', spans=[Span(text='entity', start=30, end=36, label='ENTITY', token_start=None, token_end=None, kb_id=None), Span(text='entity', start=49, end=55, label='ENTITY', token_start=None, token_end=None, kb_id=None)], tokens=None, meta={}, formatted=True),
 Example(text='This is a first sentence with new entity. This is an new entity in the 2nd sentence.', spans=[Span(text='new entity', start=30, end=40, label='ENTITY', token_start=None, token_end=None, kb_id=None), Span(text='new entity', start=53, end=63, label='ENTITY', token_start=None, token_end=None, kb_id=None)], tokens=None, meta={}, formatted=True)]